# Preprocessing

## Import

In [11]:
import pickle
%run ../scripts/save_utils.py

## Cleaning

First of all, let's look at the dataset again:

In [13]:
df = load_data('../save_files/df.pkl')

Index(['is_sarcastic', 'headline', 'article_link'], dtype='object')
